In [3]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [6]:
data_path_red_wine = "wine+quality/winequality-red.csv"
data = pd.read_csv(data_path_red_wine,sep=';')

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1245,7.4,0.55,0.19,1.80,0.082,15.0,34.0,0.99655,3.49,0.68,10.5,5
329,10.7,0.46,0.39,2.00,0.061,7.0,15.0,0.99810,3.18,0.62,9.5,5
840,11.1,0.42,0.47,2.65,0.085,9.0,34.0,0.99736,3.24,0.77,12.1,7
1488,5.6,0.54,0.04,1.70,0.049,5.0,13.0,0.99420,3.72,0.58,11.4,5
603,13.2,0.46,0.52,2.20,0.071,12.0,35.0,1.00060,3.10,0.56,9.0,6
625,6.8,0.69,0.00,5.60,0.124,21.0,58.0,0.99970,3.46,0.72,10.2,5
48,6.4,0.40,0.23,1.60,0.066,5.0,12.0,0.99580,3.34,0.56,9.2,5
1114,5.0,0.40,0.50,4.30,0.046,29.0,80.0,0.99020,3.49,0.66,13.6,6
304,8.4,0.65,0.60,2.10,0.112,12.0,90.0,0.99730,3.20,0.52,9.2,5
667,11.3,0.34,0.45,2.00,0.082,6.0,15.0,0.99880,2.94,0.66,9.2,6


In [7]:
remote_server_uri = "http://0.0.0.0:5001"
mlflow.set_tracking_uri(remote_server_uri)

In [8]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5001'

In [9]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/11/12 12:45:45 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/mnt/d/Assignment1/Assignment3.2/mlflow-test/mlruns/247104593926288002', creation_time=1731395746022, experiment_id='247104593926288002', last_update_time=1731395746022, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [18]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)

    return rmse, mae, r2

def load_data(data_path):
    data = pd.read_csv(data_path,sep=';')

    #split
    train, test = train_test_split(data)

    train_X = train.drop(["quality"], axis=1)
    test_X = test.drop(['quality'], axis=1)
    train_y = train[['quality']]
    test_y = test[['quality']]

    return train_X, train_y, test_X, test_y

def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings('ignore')
    np.random.seed(40)

    train_X, train_y, test_X, test_y = load_data(data_path_red_wine)

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_X, train_y)

        # eval metric
        predict_qualities = lr.predict(test_X)
        rmse, mae, r2 = eval_metrics(test_y, predict_qualities)

        print("ElasticNet Model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("RMSE: %s" %rmse)
        print("MAE: %s" %mae)
        print("R2: %s" %r2)

        # log params
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path_red_wine)

        print("Save to: {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, 'model')


In [19]:
train(0.5, 0.5)

ElasticNet Model (alpha=0.500000, l1_ratio=0.500000):
RMSE: 0.7931640229276851
MAE: 0.6271946374319586
R2: 0.10862644997792614
Save to: /mnt/d/Assignment1/Assignment3.2/mlflow-test/mlruns/247104593926288002/013fff3d37f946d7abb14a097d1158d9/artifacts


2024/11/12 13:40:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/12 13:40:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-ray-420 at: http://0.0.0.0:5001/#/experiments/247104593926288002/runs/013fff3d37f946d7abb14a097d1158d9.
2024/11/12 13:40:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5001/#/experiments/247104593926288002.


In [20]:
train(0.2, 0.2)

ElasticNet Model (alpha=0.200000, l1_ratio=0.200000):
RMSE: 0.7336400911821402
MAE: 0.5643841279275428
R2: 0.23739466063584158
Save to: /mnt/d/Assignment1/Assignment3.2/mlflow-test/mlruns/247104593926288002/9fc7cd6f172648d298a72b15b4cfad62/artifacts


2024/11/12 13:41:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/12 13:41:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run kindly-perch-407 at: http://0.0.0.0:5001/#/experiments/247104593926288002/runs/9fc7cd6f172648d298a72b15b4cfad62.
2024/11/12 13:41:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5001/#/experiments/247104593926288002.
